In [1]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import numpy as np
import logging
import asyncio

# Constants for configuration
DEFAULT_PRECISION = 2

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Function to read an Excel file
async def read_excel_file_async(filename: str, sheet_name: str, header_skip: int, skip_footer: int, use_cols_str: str) -> pd.DataFrame:
    try:
        use_cols = [int(col) for col in use_cols_str.split(',')]

        with pd.ExcelFile(filename) as xls:
            com_data_debit = pd.read_excel(xls, sheet_name=sheet_name, header=header_skip, skipfooter=skip_footer, usecols=use_cols)

        com_data_debit.dropna(axis=0, inplace=True)
        return com_data_debit
    except FileNotFoundError:
        logger.error(f"Error: File '{filename}' not found.")
        return None
    except (ValueError, KeyError) as e:
        logger.error(f"Invalid input: {str(e)}")
        return None

# Function to handle button click and perform data analysis
async def analyze_data_async():
    filename = await loop.run_in_executor(None, lambda: filedialog.askopenfilename(title="Select Excel File"))
    
    # Retrieve other values in the main thread
    sheet_name = await loop.run_in_executor(None, lambda: sheet_name_entry.get())
    header_skip = await loop.run_in_executor(None, lambda: header_skip_entry.get())
    skip_footer = await loop.run_in_executor(None, lambda: skip_footer_entry.get())
    use_cols_str = await loop.run_in_executor(None, lambda: use_cols_entry.get())

    com_data_debit = await read_excel_file_async(filename, sheet_name, header_skip, skip_footer, use_cols_str)

    if com_data_debit is not None:
        output_filename = f"Sub-Category-Debit-Total-{sheet_name}.xlsx"
        save_results(com_data_debit, output_filename)
        
        # Update the result label in the main thread
        root.after(0, lambda: result_label.config(text=f"Results saved to {output_filename}"))

# Create a GUI window
root = tk.Tk()
root.title("Excel Data Analysis")

# Create GUI components
file_button = tk.Button(root, text="Select Excel File", command=lambda: asyncio.ensure_future(analyze_data_async()))
file_button.pack(pady=10)

sheet_name_label = tk.Label(root, text="Sheet Name:")
sheet_name_label.pack()
sheet_name_entry = tk.Entry(root)
sheet_name_entry.pack()

header_skip_label = tk.Label(root, text="Header Skip:")
header_skip_label.pack()
header_skip_entry = tk.Entry(root)
header_skip_entry.pack()

skip_footer_label = tk.Label(root, text="Skip Footer:")
skip_footer_label.pack()
skip_footer_entry = tk.Entry(root)
skip_footer_entry.pack()

use_cols_label = tk.Label(root, text="Use Columns (comma-separated):")
use_cols_label.pack()
use_cols_entry = tk.Entry(root)
use_cols_entry.pack()

analyze_button = tk.Button(root, text="Analyze Data", command=lambda: asyncio.ensure_future(analyze_data_async()))
analyze_button.pack(pady=10)

result_label = tk.Label(root, text="")
result_label.pack()

loop = asyncio.get_event_loop()

# Start the GUI main loop
root.mainloop()


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-5' coro=<analyze_data_async() done, defined at C:\Users\DELL\AppData\Local\Temp\ipykernel_4520\426426902.py:33> exception=RuntimeError('main thread is not in main loop')>
Traceback (most recent call last):
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_4520\426426902.py", line 37, in analyze_data_async
    sheet_name = await loop.run_in_executor(None, lambda: sheet_name_entry.get())
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\anaconda3\Lib\concurrent\futures\thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_4520\426426902.py", line 37, in <lambda>
    sheet_name = await loop.run_in_executor(None, lambda: sheet_name_entry.get())
                                                          ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\